In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

In [3]:
#Загрузка данных
ratings = pd.read_csv('data/ratings.csv', parse_dates=['timestamp'])

In [4]:
#Ввиду слабого железа я не могу работать с полным датасетом, поэтому оставляю оценки только 20% пользователей
rand_userIds = np.random.choice(ratings['userId'].unique(), 
                                size=int(len(ratings['userId'].unique())*0.2), 
                                replace=False)

ratings = ratings.loc[ratings['userId'].isin(rand_userIds)]

In [5]:
ratings.sample(5)

,userId,movieId,rating,timestamp
7827833,80508,185033,2.5,1522623143
18705899,190822,2393,3.0,939980079
27557800,281116,2567,3.0,1423517707
9761287,100658,4979,4.0,1057456561
21204908,216633,2336,4.0,1192458400


'''Все жанры, представленные в описании датасета'''
genres = ['Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy',
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
          'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 
          'War', 'Western']
genres_dict = {i:[] for i in genres}

'''Применеие OneHotEncoder к столбцу genres'''
for line in movies['genres']:
    for genre in genres_dict.keys():
        genres_dict[genre].append(int(genre in line))

for genre in genres_dict.keys():
    movies[genre] = genres_dict[genre]
movies.drop(['genres'], axis=1, inplace=True) #убирается старая колонна genres за ненадобностью
movies.head()

Разбиение на тренировочную-тестовую выборку. В качестве тестовой выборки будет выступать последний фильм, чтобы не возникало ситуаций посмотренный пользователем, так как в случае случайного разбиения могут возникнуть "утечки данных". То есть может получиться, что для обучения модели мы используем данные которые появились позже тестовой выборки.

In [5]:
#Составляется набор данных по последним оценкам пользователя
ratings['rank_last'] = ratings.groupby(['userId'])['timestamp'].rank(method='first', ascending=False)

train_ratings = ratings[ratings['rank_last'] != 1].drop(['timestamp', 'rank_last'], axis=1)
test_ratings = ratings[ratings['rank_last'] == 1].drop(['timestamp', 'rank_last'], axis=1)

Основано на Implicit feedback, то есть будем смотреть на то, что пользователь взаимодействовал с фильмом, в данном случае, если пользователь поставил оценку, то ставится 1, в противном случае 0

In [6]:
#Превращение всех оценок в 1
train_ratings.loc[:, 'rating'] = 1

train_ratings.sample(5)

,userId,movieId,rating
20551373,209788,78469,1
17513365,178950,2542,1
22809301,233127,3421,1
19847304,202511,231,1
9008492,93054,4361,1


Однако в наборе данных остались только положительные примеры (пользователь взаимодействовал с фильмом), надо добавить и отрицательные.

In [7]:
class MovieLensTrainDataset(Dataset):
    """MovieLens PyTorch Dataset for Training
    
    Args:
        ratings (pd.DataFrame): Dataframe containing the movie ratings
        all_movieIds (list): List containing all movieIds
    
    """

    def __init__(self, ratings, all_movieIds):
        self.users, self.items, self.labels = self.get_dataset(ratings, all_movieIds)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings, all_movieIds):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings['userId'], ratings['movieId']))

        num_negatives = 4
        for u, i in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_movieIds)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_movieIds)
                users.append(u)
                items.append(negative_item)
                labels.append(0)

        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)

In [8]:
class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF)
    
        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items
            ratings (pd.DataFrame): Dataframe containing the movie ratings for training
            all_movieIds (list): List containing all movieIds (train + test)
    """
    
    def __init__(self, num_users, num_items, ratings, all_movieIds):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.ratings = ratings
        self.all_movieIds = all_movieIds
        
    def forward(self, user_input, item_input):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(MovieLensTrainDataset(self.ratings, self.all_movieIds),
                          batch_size=512) #, num_workers=4)

In [9]:
num_users = ratings['userId'].max()+1
num_items = ratings['movieId'].max()+1

all_movieIds = ratings['movieId'].unique()

model = NCF(num_users, num_items, train_ratings, all_movieIds)

In [10]:
trainer = pl.Trainer(max_epochs=5, 
                     enable_progress_bar=True, 
                     enable_checkpointing=True
                     )

trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 2.3 M 
1 | item_embedding | Embedding | 1.6 M 
2 | fc1            | Linear    | 1.1 K 
3 | fc2            | Linear    | 2.1 K 
4 | output         | Linear    | 33    
---------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.280    Total estimated model params size (MB)
c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:229: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve 

Training: 0it [00:00, ?it/s]

c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [11]:
# User-item pairs for testing
test_user_item_set = set(zip(test_ratings['userId'], test_ratings['movieId']))

# Dict of all items that are interacted with by each user
user_interacted_items = ratings.groupby('userId')['movieId'].apply(list).to_dict()

hits = []
for (u,i) in tqdm(test_user_item_set):
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_movieIds) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
    
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    
    if i in top10_items:
        hits.append(1)
    else:
        hits.append(0)
        
print("The Hit Ratio @ 10 is {:.2f}".format(np.average(hits)))

  0%|          | 0/70807 [00:00<?, ?it/s]

The Hit Ratio @ 10 is 0.92
